<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/yykim/Naive_Bayes_Classifier_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Naive Bayes Classifier 구현
##### Spam/Normal 분류 예시



In [3]:
import numpy as np

In [4]:
m1 = 'me free lottery'
m2 = 'free get free you'
m3 = 'you free scholarhip'
m4 = 'free to contact me'
m5 = 'you won award'
m6 = 'you ticket lottery'
labels = ['spam', 'spam', 'normal', 'normal', 'normal', 'spam']
mails = [m1, m2, m3, m4, m5, m6]

#토크나이징
for i, mail in enumerate(mails):
    mails[i] = mail.split(" ")
mails

[['me', 'free', 'lottery'],
 ['free', 'get', 'free', 'you'],
 ['you', 'free', 'scholarhip'],
 ['free', 'to', 'contact', 'me'],
 ['you', 'won', 'award'],
 ['you', 'ticket', 'lottery']]

###Class 만들기 (다중분류, sklearn 포함)

In [134]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.model_selection import GridSearchCV

class NaiveBayesClassifier_multi:
    def __init__(self, docs = [], labels = [], words=[], k = 1, package= 'manual'):
    # docs는 토큰화한 문서들을 리스트화 한 이중 리스트
        self.docs = docs
        self.labels = labels
        self.k = k #laplace smoothing을 위한 상수 k
        self.words = words
        self.package = package 
        self.total_token = []
        self.unique_token = np.array((1,1))
        self.category_name=[]
        self.classified_token=[]
        self.prior_p=[]
        self.probability_dict=[]


    def _cal_prior(self):
        #토큰 개수 세기 
        for doc in self.docs:
            for token in doc:
                self.total_token.append(token)
        self.unique_token = np.unique(self.total_token)        
        #카테고리별 확률 구하기
        self.category_name = np.unique(labels)

        count_labels=[]
        for name in self.category_name:
            tmp = []
            for idx, doc in enumerate(self.docs):
                if self.labels[idx] == name:
                    tmp.extend(doc)
            self.classified_token.append(tmp)
            count_labels.append(len(tmp))
        self.prior_p = list(np.array(count_labels)/len(self.total_token))
        return dict(zip(list(self.category_name),self.prior_p))

    def _cal_posterior(self):
        self._cal_prior()
       #각각의 카테고리에서 토큰이 나온 횟수세기        
        ls_n_classified_token=[]            
        for classfiedtoken in self.classified_token:
            n_classified_token=[]
            for token in self.unique_token:
                tmp = classfiedtoken.count(token)
                n_classified_token.append(tmp)
            ls_n_classified_token.append(n_classified_token)

        for i, v in enumerate(ls_n_classified_token):
            for idx, w in enumerate(v):
                ls_n_classified_token[i][idx] = np.log((w+1*self.k)/(sum(v)+2*self.k))
            tmp = dict(zip(list(self.unique_token),ls_n_classified_token[i]))
            self.probability_dict.append(tmp)
        return self.probability_dict
        
    def _use_handmadeone(self):
        self._cal_prior()
        self._cal_posterior()
        p_list= []
        for idx, p in enumerate(self.prior_p): 
            tmp = np.log(p)
            for word in self.words:
                tmp += self.probability_dict[idx][word]
            tmp = np.exp(tmp)
            p_list.append(tmp) 

        return self.category_name[p_list.index(max(p_list))]

    def _use_plain_sklearn(self):
        text_clf = Pipeline([('vect', CountVectorizer()),
                            ('tfidf', TfidfTransformer()),
                            ('clf', MultinomialNB()), ])
        og_docs = [" ".join(i) for i in self.docs]
        self.docs = og_docs
        text_clf = text_clf.fit(self.docs, self.labels)

        self.words = [" ".join(self.words)]
        return text_clf.predict(self.words)

    def _use_gs_sklearn(self, parameters_dict):
        text_clf = Pipeline([('vect', CountVectorizer()),
                            ('tfidf', TfidfTransformer()),
                            ('clf', MultinomialNB()), ])
        og_docs = [" ".join(i) for i in self.docs]
        self.docs = og_docs
        text_clf = text_clf.fit(self.docs, self.labels)
        
        # parameters_dict = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False)}
        gs_clf = GridSearchCV(text_clf, parameters_dict, n_jobs=-1, verbose=2)
        gs_clf = gs_clf.fit(self.docs, self.labels)

        self.words = [" ".join(self.words)]
        return gs_clf.best_estimator_.get_params(), gs_clf.predict(self.words)

    def classify(self):
        if self.package == 'manual':
            self._cal_prior()
            self._cal_posterior()
            return self._use_handmadeone()

        elif self.package == 'sklearn':
            return self._use_plain_sklearn()

        elif self.package == 'sklearn_gs':
            from ast import literal_eval
            parameters_dict = input("조정할 파라미터 딕셔너리를 입력해주세요: ")
            parameters_dict = literal_eval(parameters_dict) 
            return self._use_gs_sklearn(parameters_dict)

        else:
            print("지원하지 않는 방식입니다.")



In [135]:
nbc = NaiveBayesClassifier_multi(docs=mails, labels=labels, words=['lottery', 'free'], k=0.5)


In [136]:
nbc.classify()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in log


'spam'

In [137]:
nbc1 = NaiveBayesClassifier_multi(docs=mails, labels=labels, words=['lottery', 'free'], k=0.5, package='sklearn')


In [138]:
nbc1.classify()

array(['spam'], dtype='<U6')

In [139]:
nbc2 = NaiveBayesClassifier_multi(docs=mails, labels=labels, words=['lottery', 'free'], k=0.5, package='sklearn_gs')

In [131]:
nbc2.classify() # 데이터 부족으로 오류 생김

조정할 파라미터 딕셔너리를 입력해주세요: {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False)}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


ValueError: ignored

In [132]:
#데이터 늘린 후 다시시도

m1 = 'me free lottery'
m2 = 'free get free you'
m3 = 'you free scholarhip'
m4 = 'free to contact me'
m5 = 'you won award'
m6 = 'you ticket lottery'
labels = ['spam', 'spam', 'normal', 'normal', 'normal', 'spam', 'spam', 'spam', 'normal', 'normal', 'normal', 'spam']
mails = [m1, m2, m3, m4, m5, m6, m1, m2, m3, m4, m5, m6]

#토크나이징
for i, mail in enumerate(mails):
    mails[i] = mail.split(" ")
mails

[['me', 'free', 'lottery'],
 ['free', 'get', 'free', 'you'],
 ['you', 'free', 'scholarhip'],
 ['free', 'to', 'contact', 'me'],
 ['you', 'won', 'award'],
 ['you', 'ticket', 'lottery'],
 ['me', 'free', 'lottery'],
 ['free', 'get', 'free', 'you'],
 ['you', 'free', 'scholarhip'],
 ['free', 'to', 'contact', 'me'],
 ['you', 'won', 'award'],
 ['you', 'ticket', 'lottery']]

In [133]:
nbc2 = NaiveBayesClassifier_multi(docs=mails, labels=labels, words=['lottery', 'free'], k=0.5, package='sklearn_gs')
nbc2.classify()

조정할 파라미터 딕셔너리를 입력해주세요: {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False)}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:    0.9s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.9s finished


({'clf': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
  'clf__alpha': 1.0,
  'clf__class_prior': None,
  'clf__fit_prior': True,
  'memory': None,
  'steps': [('vect',
    CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                    dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                    lowercase=True, max_df=1.0, max_features=None, min_df=1,
                    ngram_range=(1, 1), preprocessor=None, stop_words=None,
                    strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                    tokenizer=None, vocabulary=None)),
   ('tfidf',
    TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
   ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
  'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
  'tfidf__norm': 'l2',
  'tfidf__smooth_idf': True,
  'tfidf__sublinear_tf': False,
  'tfidf__use_idf': True,
  'v